# About this notebook..
this is copy of [Sensory_GRU_Sweep_condition_labeled.ipynb](https://github.com/chhyyi/aiffelthon/blob/main/lms/Sensory_GRU_Sweep_condition_labeled.ipynb)

## Changelog:
- train-test split is omitted. instead, it loads train, test dataset respectively.

In [ ]:
!pip3 install wandb

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import os
import wandb
from wandb.keras import WandbCallback

In [7]:
def train():
    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    import matplotlib.pyplot as plt
    import numpy as np
    import wandb
    from wandb.keras import WandbCallback

    def seq_acc(y_true, y_pred):
        y_bin=np.zeros_like(y_pred)
        for i, dd in enumerate(y_bin):
            for j in range(len(dd)):
                pred=y_pred[i][j]
                if pred>=0.5:
                    y_bin[i][j]=1
                else:
                    y_bin[i][j]=0

        predict_true = (y_true == y_bin)

        try:
            score = np.average(np.average(predict_true))
        except ValueError:
            score = mean_squared_error(y_true, y_bin)
        return score

    def my_seq_acc(y_true, y_pred):
        score = tf.py_function(func=seq_acc, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_seq_acc') # tf 2.x
        return score


    class MySeqAccCallback(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.seq_acc_history = []

        def on_epoch_end(self, epochs, logs=None):
            y_pred=self.model.predict(X_test)
            print('sequence accuracy is {}'.format(seq_acc(y_test, y_pred)))
            self.seq_acc_history.append(seq_acc(y_test, y_pred))

    default_config={
        'target_type':'실측',
        'RNN_type':'GRU',
        'seq_field':72,
        'stride_inside_seq':9,
        'stride_between_seqs':2,
        'learning_rate':0.01,
        'epochs':20,
        'batch_size':64,
        'unit_RNN0':64}
    ######### Wandb.init() ##########
    wandb.init(config = default_config)
    
    def not_bin_in_occurence(x):
        if x==1 or x==0:
            return x
        else:
            #print('exceptional value(not 0 or 1) found. replaced by near one.')
            if x>=0.5:
                return 1
            else:
                return 0
            

    locations=['거문도', '울산', '거제도', '통영', '추자도']
    if wandb.config.target_type=='실측':
        df_merged_train=pd.read_csv("sample_data/class1_obs_train_refined.csv")
        df_merged_test=pd.read_csv("sample_data/class1_obs_test_refined.csv")
        df_merged_train['적조발생(실측)']=df_merged_train['적조발생(실측)'].apply(not_bin_in_occurence)
        df_merged_test['적조발생(실측)']=df_merged_test['적조발생(실측)'].apply(not_bin_in_occurence)
    elif wandb.config.target_type=='조건':
        df_merged_train=pd.read_csv("sample_data/class1_condition_train_refined.csv")
        df_merged_test=pd.read_csv("sample_data/class1_condition_test_refined.csv")
        df_merged_train['적조발생(조건)']=df_merged_train['적조발생(조건)'].apply(not_bin_in_occurence)
        df_merged_test['적조발생(조건)']=df_merged_test['적조발생(조건)'].apply(not_bin_in_occurence)


    if df_merged_train.columns[0]=='Unnamed: 0':
        df_merged_train = df_merged_train.iloc[:, 1:]
        print('drop "Unnamed: 0" column')

    if df_merged_test.columns[0]=='Unnamed: 0':
        df_merged_test = df_merged_test.iloc[:, 1:]
        print('drop "Unnamed: 0" column')
        
    print('dataset loaded. Generating sequences')
    
    #train set
    df_merged = df_merged_train
    seq_length=wandb.config.seq_field//wandb.config.stride_inside_seq
    len_ds=len(df_merged)

    seqs_idx=[]

    start_idx=0
    while start_idx<=len_ds-wandb.config.seq_field:
        seqs_idx.append(list(range(start_idx, start_idx + wandb.config.seq_field, wandb.config.stride_inside_seq
    )))
        start_idx+=wandb.config.stride_between_seqs

    df_merged.reset_index(inplace=True, drop=True)
    print('Any missing values exist:', df_merged.isna().all().all())

    ds_train_cols=df_merged
    ds_train_cols.reset_index(inplace=True, drop=True)
    print('train dataset columns:',ds_train_cols.columns)

    seq_dataset=np.zeros([len(seqs_idx), len(seqs_idx[0]), len(ds_train_cols.columns)])

    for i, seq in enumerate(seqs_idx):
        for j, row_number in enumerate(seq):
            seq_dataset[i, j]=ds_train_cols.loc[row_number].to_numpy()
    train_xy=seq_dataset
            
    #test set
    df_merged=df_merged_test
        
    seq_length=wandb.config.seq_field//wandb.config.stride_inside_seq
    len_ds=len(df_merged)

    seqs_idx=[]

    start_idx=0
    while start_idx<=len_ds-wandb.config.seq_field:
        seqs_idx.append(list(range(start_idx, start_idx + wandb.config.seq_field, wandb.config.stride_inside_seq
    )))
        start_idx+=wandb.config.stride_between_seqs

    df_merged.reset_index(inplace=True, drop=True)
    print('Any missing values exist:', df_merged.isna().all().all())

    ds_train_cols=df_merged
    ds_train_cols.reset_index(inplace=True, drop=True)
    print('train dataset columns:',ds_train_cols.columns)

    seq_dataset_test=np.zeros([len(seqs_idx), len(seqs_idx[0]), len(ds_train_cols.columns)])

    for i, seq in enumerate(seqs_idx):
        for j, row_number in enumerate(seq):
            seq_dataset_test[i, j]=ds_train_cols.loc[row_number].to_numpy()

    test_xy=seq_dataset_test
    
    #split labels
    X_train=train_xy[:,:,0:-1]
    y_train=train_xy[:,:,-1]
    X_test=test_xy[:,:,0:-1]
    y_test=test_xy[:,:,-1]

    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape,'\n\n')
    if wandb.config.RNN_type=='GRU':        
        model = keras.Sequential([
            keras.Input(shape=(seq_length, 25)),
            keras.layers.GRU(wandb.config.unit_RNN0),
            keras.layers.Dense(seq_length, activation="sigmoid"),
        ])
    elif wandb.config.RNN_type=='LSTM':
        model = keras.Sequential([
            keras.Input(shape=(seq_length, 25)),
            keras.layers.LSTM(wandb.config.unit_RNN0),
            keras.layers.Dense(seq_length, activation="sigmoid"),
        ])
    optimizer=keras.optimizers.Adam(learning_rate=wandb.config.learning_rate)

    model.compile(optimizer=optimizer, loss="binary_crossentropy")
    my_callback=MySeqAccCallback()
    ######### WandbCallback ##########
    history=model.fit(X_train, y_train,
            batch_size=wandb.config.batch_size,
            epochs=wandb.config.epochs, 
            validation_data=(X_test, y_test),
            callbacks=[WandbCallback(training_data = (X_train, y_train),
                                     validation_data = (X_test, y_test)), my_callback])
    y_pred=model.predict(X_test)
    
    ######### Wandb.log() ##########
    wandb.log({"ValidationAcc":seq_acc(y_test, y_pred), "best_val_acc":np.max(my_callback.seq_acc_history)})

In [3]:
wandb.login()

wandb: Currently logged in as: chhyyi. Use `wandb login --relogin` to force relogin


True

In [20]:
sweep_config = {
    "name": "sweep",
    "metric": {
        "goal": "minimize",
        "name": "best_val_loss"
    },
    "method": "random",
    "parameters": {
        "RNN_type":{
            "values":['GRU', 'LSTM']
        },
        "target_type":{
            "values":['실측', '조건']
        },
        "learning_rate":{"values":[0.0002, 0.001, 0.0032, 0.01]},
        "unit_RNN0":{"values":[32,64,128]},
        "stride_inside_seq":{"values":[3,9]}
    }
    }

In [ ]:
sweep_id = wandb.sweep(sweep_config)

# run the sweep
wandb.agent(sweep_id,
            function=train,
            entity = 'chhyyi',
            project = 'redzone_RNN_class1_sweep2')

Create sweep with ID: lgpikngz
Sweep URL: https://wandb.ai/chhyyi/redzone_RNN_class1_sweep2/sweeps/lgpikngz


wandb: Agent Starting Run: oqde172s with config:
wandb: 	RNN_type: LSTM
wandb: 	learning_rate: 0.001
wandb: 	stride_inside_seq: 3
wandb: 	target_type: 조건
wandb: 	unit_RNN0: 64


drop "Unnamed: 0" column
drop "Unnamed: 0" column
dataset loaded. Generating sequences
Any missing values exist: False
train dataset columns: Index(['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '풍속(m/s).1',
       '풍향(deg).1', '기온(°C).1', '수온(°C).1', '강수량(mm).1', '풍속(m/s).2',
       '풍향(deg).2', '기온(°C).2', '수온(°C).2', '강수량(mm).2', '풍속(m/s).3',
       '풍향(deg).3', '기온(°C).3', '수온(°C).3', '강수량(mm).3', '풍속(m/s).4',
       '풍향(deg).4', '기온(°C).4', '수온(°C).4', '강수량(mm).4', '적조발생(조건)'],
      dtype='object')
Any missing values exist: False
train dataset columns: Index(['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '풍속(m/s).1',
       '풍향(deg).1', '기온(°C).1', '수온(°C).1', '강수량(mm).1', '풍속(m/s).2',
       '풍향(deg).2', '기온(°C).2', '수온(°C).2', '강수량(mm).2', '풍속(m/s).3',
       '풍향(deg).3', '기온(°C).3', '수온(°C).3', '강수량(mm).3', '풍속(m/s).4',
       '풍향(deg).4', '기온(°C).4', '수온(°C).4', '강수량(mm).4', '적조발생(조건)'],
      dtype='object')
(25131, 24, 25) (27940, 24, 25) (25131, 24) (27940, 2

INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221120_192600-oqde172s/files/model-best/assets


INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221120_192600-oqde172s/files/model-best/assets
wandb: Adding directory to artifact (/aiffel/aiffel/aiffelthon/wandb/run-20221120_192600-oqde172s/files/model-best)... Done. 0.0s


...

wandb: Agent Starting Run: 3gnwtl1u with config:
wandb: 	RNN_type: LSTM
wandb: 	learning_rate: 0.01
wandb: 	stride_inside_seq: 3
wandb: 	target_type: 조건
wandb: 	unit_RNN0: 64


drop "Unnamed: 0" column
drop "Unnamed: 0" column
dataset loaded. Generating sequences
Any missing values exist: False
train dataset columns: Index(['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '풍속(m/s).1',
       '풍향(deg).1', '기온(°C).1', '수온(°C).1', '강수량(mm).1', '풍속(m/s).2',
       '풍향(deg).2', '기온(°C).2', '수온(°C).2', '강수량(mm).2', '풍속(m/s).3',
       '풍향(deg).3', '기온(°C).3', '수온(°C).3', '강수량(mm).3', '풍속(m/s).4',
       '풍향(deg).4', '기온(°C).4', '수온(°C).4', '강수량(mm).4', '적조발생(조건)'],
      dtype='object')
Any missing values exist: False
train dataset columns: Index(['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '풍속(m/s).1',
       '풍향(deg).1', '기온(°C).1', '수온(°C).1', '강수량(mm).1', '풍속(m/s).2',
       '풍향(deg).2', '기온(°C).2', '수온(°C).2', '강수량(mm).2', '풍속(m/s).3',
       '풍향(deg).3', '기온(°C).3', '수온(°C).3', '강수량(mm).3', '풍속(m/s).4',
       '풍향(deg).4', '기온(°C).4', '수온(°C).4', '강수량(mm).4', '적조발생(조건)'],
      dtype='object')
(25131, 24, 25) (27940, 24, 25) (25131, 24) (27940, 2

INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221120_233023-3gnwtl1u/files/model-best/assets


INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221120_233023-3gnwtl1u/files/model-best/assets
wandb: Adding directory to artifact (/aiffel/aiffel/aiffelthon/wandb/run-20221120_233023-3gnwtl1u/files/model-best)... Done. 0.0s


sequence accuracy is 0.9989620615604867
Epoch 2/20
393/393 [==============================] - 2s 5ms/step - loss: 0.0125 - val_loss: 0.0075
sequence accuracy is 0.9988129324743498
Epoch 3/20
393/393 [==============================] - 2s 5ms/step - loss: 0.0127 - val_loss: 0.0075
sequence accuracy is 0.9989620615604867
Epoch 4/20
393/393 [==============================] - 2s 5ms/step - loss: 0.0117 - val_loss: 0.0077
sequence accuracy is 0.9989620615604867
Epoch 5/20
393/393 [==============================] - 2s 5ms/step - loss: 0.0103 - val_loss: 0.0078


In [8]:
sweep_config = {
    "name": "increase_epoch_of_volcanic13_with_best_val_acc",
    "metric": {
        "goal": "minimize",
        "name": "best_val_loss"
    },
    "method": "grid",
    "parameters": {
        "RNN_type":{
            "values":['LSTM']
        },
        "target_type":{
            "values":['실측']
        },
        "epochs":{"values":[60]},
        "learning_rate":{"values":[0.001]},
        "unit_RNN0":{"values":[128]},
        "stride_inside_seq":{"values":[9]}
    }
    }

In [9]:
sweep_id = wandb.sweep(sweep_config)

# run the sweep
wandb.agent(sweep_id,
            function=train,
            entity = 'chhyyi',
            project = 'redzone_RNN_class1_sweep2')

Create sweep with ID: zrve5kds
Sweep URL: https://wandb.ai/chhyyi/redzone_RNN_class1_sweep2/sweeps/zrve5kds


wandb: Agent Starting Run: 216yg8bh with config:
wandb: 	RNN_type: LSTM
wandb: 	epochs: 60
wandb: 	learning_rate: 0.001
wandb: 	stride_inside_seq: 9
wandb: 	target_type: 실측
wandb: 	unit_RNN0: 128


drop "Unnamed: 0" column
drop "Unnamed: 0" column
dataset loaded. Generating sequences
Any missing values exist: False
train dataset columns: Index(['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '풍속(m/s).1',
       '풍향(deg).1', '기온(°C).1', '수온(°C).1', '강수량(mm).1', '풍속(m/s).2',
       '풍향(deg).2', '기온(°C).2', '수온(°C).2', '강수량(mm).2', '풍속(m/s).3',
       '풍향(deg).3', '기온(°C).3', '수온(°C).3', '강수량(mm).3', '풍속(m/s).4',
       '풍향(deg).4', '기온(°C).4', '수온(°C).4', '강수량(mm).4', '적조발생(실측)'],
      dtype='object')
Any missing values exist: False
train dataset columns: Index(['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '풍속(m/s).1',
       '풍향(deg).1', '기온(°C).1', '수온(°C).1', '강수량(mm).1', '풍속(m/s).2',
       '풍향(deg).2', '기온(°C).2', '수온(°C).2', '강수량(mm).2', '풍속(m/s).3',
       '풍향(deg).3', '기온(°C).3', '수온(°C).3', '강수량(mm).3', '풍속(m/s).4',
       '풍향(deg).4', '기온(°C).4', '수온(°C).4', '강수량(mm).4', '적조발생(실측)'],
      dtype='object')
(25131, 8, 25) (25130, 8, 25) (25131, 8) (25130, 8) 


INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221121_042047-216yg8bh/files/model-best/assets


INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221121_042047-216yg8bh/files/model-best/assets
wandb: Adding directory to artifact (/aiffel/aiffel/aiffelthon/wandb/run-20221121_042047-216yg8bh/files/model-best)... Done. 0.0s


...

INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221121_042047-216yg8bh/files/model-best/assets


INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221121_042047-216yg8bh/files/model-best/assets
wandb: Adding directory to artifact (/aiffel/aiffel/aiffelthon/wandb/run-20221121_042047-216yg8bh/files/model-best)... Done. 0.0s


sequence accuracy is 0.985908276959809
Epoch 10/60
393/393 [==============================] - 2s 5ms/step - loss: 0.0126 - val_loss: 0.0512
sequence accuracy is 0.9860873458018304
Epoch 11/60
393/393 [==============================] - 2s 5ms/step - loss: 0.0116 - val_loss: 0.0455
sequence accuracy is 0.984440907282133
Epoch 12/60
393/393 [==============================] - 2s 5ms/step - loss: 0.0086 - val_loss: 0.0593
sequence accuracy is 0.9838738559490648
Epoch 13/60
393/393 [==============================] - 2s 5ms/step - loss: 0.0120 - val_loss: 0.0485
sequence accuracy is 0.9873806207719856
Epoch 14/60
393/393 [==============================] - 2s 5ms/step - loss: 0.0052 - val_loss: 0.0521
sequence accuracy is 0.9878481894150418
Epoch 15/60
393/393 [==============================] - 2s 5ms/step - loss: 0.0050 - val_loss: 0.0520
sequence accuracy is 0.9885395941106248
Epoch 16/60
393/393 [==============================] - 2s 5ms/step - loss: 0.0068 - val_loss: 0.0538
sequence accura

ValidationAcc,▁
best_val_acc,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▃▃▂▂▂▁▁▂▂▂▂▂▂▂▃▂▂▂▂▃▃▂▂▂▄▃▃▄▄▆▃▃▄▄▄▄▄▃
ValidationAcc,0.98322
best_epoch,8
best_val_acc,0.99087
best_val_loss,0.04471
epoch,59
loss,0.0035


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [10]:
wandb.finish()